# Load Modules and Database

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as aniplt

In [ ]:
database = "../database/prisoners_dilemma.json"
with open(database, "r") as database_file:
    data = pd.read_json(database_file)

In [ ]:
def export_graph(self, animation_data: aniplt.ArtistAnimation, round: int) -> None:
    """
    This function saves the data into a file by using the ffmpeg library.
    It has been seperated from complete program so that we can call it using the process library and make the operation fast.

    Args:
        animation_data (ArtistAnimation): A matplotlib based artist animation.
        round (int): An integer value depicting the current round for which we are saving the data.

    Returns:
        None (The operation writes data into a file.)
    """
    animation_data.save(
        f"./visuals/{self.game_type}/round{round}.mp4",
        writer=aniplt.FFMpegWriter(fps=25),
    )

def analyse_global_history(self) -> None:
    """
    This function analyzes the global history and generates pre-programmed graphical insights.
    The function is isolated to ensure maximal parallelization and isolation from rest of program.

    Args:
        None
    Returns:
        None
    """
    print("Starting analysis for global history.")

    # Plotting the progress of each player in a bar chart animation.
    total_rounds = self.configurations[self.game_type]["fixture_settings"]["rounds"]
    # Setup a scoreboard for the analysis.
    self.analysis_scoreboard = {}
    for player in self.configurations[self.game_type]["players"]:
        self.analysis_scoreboard[player] = 0
    
    process_store = {}
    for round in range(total_rounds):
        figure, ax = plt.subplots()
        figure.set_figheight(10)
        figure.set_figwidth(15)
        ax.set_title(f"Round: {round}")
        ax.set_xlabel("Points")
        ax.set_ylabel("Players")
        artists = []

        # For each fixture extract game moves and generate required plots.
        fixture_moves  = [fixture["moves_data"] for fixture in self.global_history if fixture["round_id"] == round]
        for game_moves in fixture_moves:
            for move_data in game_moves:
                players = list(move_data.keys())
                player_moves = list(move_data.values())

                # Get the scores for the above players.
                player1_score, player2_score = self.score_moves(
                    player1_move=player_moves[0], player2_move=player_moves[0]
                )
                # Add the scores to scoreboard.
                self.scoreboard[players[0]] += player1_score
                self.scoreboard[players[1]] += player2_score

                # We need to score each move in the moves data.
                container = ax.barh(
                    list(self.analysis_scoreboard.keys()),
                    list(self.analysis_scoreboard.values()),
                )
                artists.append(container)
                del container
                del players
                del player_moves

        ani = aniplt.ArtistAnimation(fig=figure, artists=artists, interval=1)
        if self.configurations[f"{self.game_type}"]["writer_parallelization"] != 0:
            process_store[round] = multiprocessing.Process(
                target=self.export_graph, args=(ani, round)
            )
            process_store[round].start()
        else:
            print(f"Starting the analysis plotters for the round: {round}")
            self.export_graph(animation_data=ani, round=round)

    if self.configurations[f"{self.game_type}"]["writer_parallelization"] != 0:
        print("Waiting for analysis plotters to join back.")
        for round in process_store:
            process_store[round].join()

    return None
